In [25]:
import pandas as pd
import numpy as np
from glob import glob

from PIL import Image
import cv2
from tqdm import tqdm

import os
import json
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt

In [7]:
data_path = 'data/'
train_path = data_path + 'train'
test_path = data_path + 'test'

train_folders = sorted(glob(train_path + '/*'), key = lambda x : int(x.split('file_')[-1]))
test_folders = sorted(glob(test_path + '/*'), key = lambda x : int(x.split('file_')[-1]))

In [17]:
new_image_directory = data_path + 'new_images/'
new_train_image_directory = new_image_directory + 'train'
new_test_image_directory  = new_image_directory + 'test'

action_information = pd.read_csv(data_path + 'action_information.csv')
sample_submission = pd.read_csv(data_path + 'sample_submission.csv')

In [14]:
classes = pd.get_dummies(action_information[['Label']], columns=['Label']).to_numpy()
classes

array([[1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]], dtype=uint8)

In [18]:
def make_new_dir(path):
    if os.path.isdir(path) == False:
        os.makedirs(path)

make_new_dir(new_image_directory)
make_new_dir(new_train_image_directory)
make_new_dir(new_test_image_directory)

In [75]:
train_directories = np.array(sorted(glob(train_path + '/*'), key = lambda x : int(x.split('/')[-1].split('_')[-1])))

for train_directory in tqdm(train_directories, total = len(train_directories)):
    file_name = train_directory.split('\\')[-1]
    make_new_dir(new_train_image_directory + '\\' + file_name)
    
    image_paths = sorted(glob(train_directory + '\*jpg'), key = lambda x : int(x.split('\\')[-1].replace('.jpg','')))
    json_path = glob(train_directory + '\*.json')[0]
    
    js = json.load(open(json_path))
    target = js.get('action')
    target = classes[target]
    bounding_boxes = js.get('sequence').get('bounding_box')
    bounding_boxes = [(float(a), float(b), float(c), float(d)) for a, b, c, d in bounding_boxes]
    
    for image_path, bounding_box in zip(image_paths, bounding_boxes):
        image = Image.open(image_path)
        image = image.crop(bounding_box)
        image = image.resize((224, 224))
        image.save(new_train_image_directory + image_path.split('/train')[1])

100%|████████████████████████████████████████████████████████████████████████████████| 142/142 [24:27<00:00, 10.33s/it]


In [76]:
test_directoires = np.array(sorted(glob(test_path + '/*'), key = lambda x : int(x.split('/')[-1].split('_')[-1])))

for test_directory in tqdm(test_directoires, total = len(test_directoires)):
    file_name = test_directory.split('\\')[-1]
    make_new_dir(new_test_image_directory + '\\' + file_name)
    
    image_paths = sorted(glob(test_directory + '\*jpg'), key = lambda x : int(x.split('\\')[-1].replace('.jpg','')))
    json_path = glob(test_directory + '\*.json')[0]
    
    js = json.load(open(json_path))
    target = js.get('action')
    target = classes[target]
    bounding_boxes = js.get('sequence').get('bounding_box')
    bounding_boxes = [(float(a), float(b), float(c), float(d)) for a, b, c, d in bounding_boxes]
    
    for image_path, bounding_box in zip(image_paths, bounding_boxes):
        image = Image.open(image_path)
        image = image.crop(bounding_box)
        image = image.resize((224, 224))
        image.save(new_test_image_directory + image_path.split('/test')[1])

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [07:47<00:00, 10.40s/it]


In [26]:
import pandas as pd
import numpy as np
from glob import glob

from PIL import Image
import cv2
from tqdm import tqdm

import os
import shutil
import json

import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import albumentations as A
from efficientnet_pytorch import EfficientNet

import os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold

import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(42)

In [79]:
data_path = 'data'
train_path = data_path + '/train'
train_image_path = data_path + '/new_images/train'
action_information = pd.read_csv(data_path + '/action_information.csv')

In [83]:
train_folders = sorted(glob(train_path + '/*'), key = lambda x : int(x.split('\\file_')[-1]))
train_img_folders = sorted(glob(train_image_path + '/*'), key = lambda x : int(x.split('\\file_')[-1]))

In [86]:
imgs = []

for i in train_img_folders:
    train_imgs = sorted(glob(i+"/*.jpg"), key = lambda x : int(x.split('.jpg')[0].split("\\")[-1]))
    for j in train_imgs[25:45]:
        imgs.append(j)
    for j in train_imgs[-45:-25]:
        imgs.append(j)

In [91]:
answers = []
for train_folder in train_folders :
    json_path = glob(train_folder + '\*.json')[0]
    js = json.load(open(json_path))
    cat = js.get('action')
    answers.append([train_folder.replace(data_path,''),cat])

answers = pd.DataFrame(answers, columns = ['train_path','answer'])
answers
answers["answer"][33] = 5

In [92]:
labels = np.zeros(len(imgs))
for i in tqdm(range(answers.shape[0])):
    for j in range(len(imgs)):
        if answers["train_path"][i] in imgs[j]:
            labels[j] = answers["answer"][i]

,train_path,answer
0,/train\file_0,5
1,/train\file_1,3
2,/train\file_2,3
3,/train\file_3,3
4,/train\file_4,2
...,...,...
137,/train\file_137,2
138,/train\file_138,1
139,/train\file_139,1
140,/train\file_140,1


In [96]:
labels = np.zeros(len(imgs))
for i in tqdm(range(answers.shape[0])):
    for j in range(len(imgs)):
        if answers["train_path"][i] in imgs[j]:
            labels[j] = answers["answer"][i]

100%|████████████████████████████████████████████████████████████████████████████████| 142/142 [00:02<00:00, 66.54it/s]


In [28]:
device = torch.device("cuda:0")
dropout_rate = 0.1
class_num = 6
learning_rate = 1e-4
BATCH_SIZE = 16
EPOCHS = 25
MODELS = 'efficientnet-b0'
save_path = f"models/Final_{EPOCHS}"
FOLDS = 5

In [31]:
imgs = np.array(imgs)
labels = np.array(labels)

from sklearn.utils import shuffle
imgs, labels = shuffle(imgs, labels, random_state=42)

albumentations_transform = A.Compose([
    A.RandomGamma(),
    A.ShiftScaleRotate(),
    A.GaussianBlur(),
    A.GaussNoise()
])

In [23]:
class CustomDataset(Dataset):
    def __init__(self, imgs, labels = None, transformer = None, mode = "train"):
        self.imgs = imgs
        self.transformer = transformer
        self.mode = mode
        if self.mode == "train":
            self.labels = labels
    
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, i):
        img = cv2.imread(self.imgs[i]).astype(np.float32) / 255
        img = cv2.resize(img, dsize = (224, 224))
        if self.mode == "train":
            if self.transformer != None:
                img = self.transformer(image = img)
                img = np.transpose(img["image"], (2, 0, 1))
            else:
                img = np.transpose(img["image"], (2, 0, 1))
            return {
                "img" : torch.tensor(img, dtype = torch.float32),
                "label" : torch.tensor(self.labels[i], dtype = torch.long)
            }
        else:
            img = np.transpose(img, (2, 0, 1))
            return {
                "img" : torch.tensor(img, dtype = torch.float32)
            }

In [104]:
def train_step(batch_item, epoch, batch, training):
    img = batch_item['img'].to(device)
    label = batch_item['label'].to(device)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        return loss
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label)
        
        return loss

In [113]:
skf = StratifiedKFold(n_splits = FOLDS, random_state = 42, shuffle = True)

n_iter = 0

for train_idx, val_idx in skf.split(imgs, labels):
    model = EfficientNet.from_pretrained(MODELS, num_classes=class_num, advprop = True)
    model._dropout.p = dropout_rate
    model = model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
    criterion = nn.CrossEntropyLoss()
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience = 5)
    
    n_iter += 1
    
    train_dataset = CustomDataset(imgs[train_idx], labels[train_idx], transformer= albumentations_transform)
    val_dataset = CustomDataset(imgs[val_idx], labels[val_idx], transformer= albumentations_transform)
    
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle= True)
    val_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle= True)
    
    loss_plot, val_loss_plot = [], []
    
    for epoch in range(EPOCHS):
        total_loss, total_val_loss = 0, 0
        
        tqdm_dataset = tqdm(enumerate(train_dataloader))
        training = True
        for batch, batch_item in tqdm_dataset:
            batch_loss = train_step(batch_item, epoch, batch, training)
            total_loss += batch_loss
            
            tqdm_dataset.set_postfix({
                'Epoch' : epoch + 1,
                'Loss' : '{:06f}'.format(batch_loss.item()),
                'Total Loss' : '{:06f}'.format(total_loss / (batch + 1))
            })
        loss_plot.append(total_loss / (batch + 1))
    
        tqdm_dataset = tqdm(enumerate(val_dataloader))
        training = False
        for batch, batch_item in tqdm_dataset:
                batch_loss = train_step(batch_item, epoch, batch, training)
                total_val_loss += batch_loss

                tqdm_dataset.set_postfix({
                    'Epoch': epoch + 1,
                    'Val Loss': '{:06f}'.format(batch_loss.item()),
                    'Total Val Loss' : '{:06f}'.format(total_val_loss/(batch+1))
                })
        val_loss_plot.append(total_val_loss/(batch+1))
        scheduler.step(total_val_loss/(batch+1))
        if min(val_loss_plot) == val_loss_plot[-1]:
            torch.save(model, save_path + f"_{n_iter}.pt")
            print("## Model Save")
    break

Loaded pretrained weights for efficientnet-b0


284it [00:57,  4.94it/s, Epoch=1, Loss=1.374896, Total Loss=1.524132]
284it [00:23, 11.94it/s, Epoch=1, Val Loss=1.402359, Total Val Loss=1.061948]


## Model Save


284it [00:57,  4.94it/s, Epoch=2, Loss=1.246144, Total Loss=1.042478]
284it [00:24, 11.36it/s, Epoch=2, Val Loss=0.877390, Total Val Loss=0.943318]


## Model Save


...

## Model Save


284it [00:54,  5.20it/s, Epoch=22, Loss=0.633251, Total Loss=0.411683]
284it [00:22, 12.76it/s, Epoch=22, Val Loss=0.175138, Total Val Loss=0.358472]
284it [00:55,  5.12it/s, Epoch=23, Loss=0.494617, Total Loss=0.423848]
284it [00:22, 12.69it/s, Epoch=23, Val Loss=0.501489, Total Val Loss=0.333511]


## Model Save


284it [00:54,  5.17it/s, Epoch=24, Loss=0.155111, Total Loss=0.397514]
284it [00:22, 12.73it/s, Epoch=24, Val Loss=0.282000, Total Val Loss=0.337982]
284it [00:55,  5.15it/s, Epoch=25, Loss=0.462534, Total Loss=0.386024]
284it [00:22, 12.70it/s, Epoch=25, Val Loss=0.092500, Total Val Loss=0.289978]

## Model Save


In [3]:
import pandas as pd
import numpy as np
from glob import glob

from PIL import Image
import cv2
from tqdm import tqdm

import os
import shutil
import json

import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
import albumentations as A
import timm

import os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

In [4]:
MODELS = 'efficientnet-b0'
save_path = f"models/Final"
device = torch.device("cuda:0")
class_num = 6
FOLDS = 1

In [5]:
data_path = 'data'
test_path = data_path + '/test'
test_img_path = data_path + '/new_images/test'
hand_gesture = pd.read_csv(data_path + '/action_information.csv')

In [6]:
test_folders = sorted(glob(test_path + '/*'), key = lambda x : int(x.split('\\file_')[-1]))
test_img_folders = sorted(glob(test_img_path + '/*'), key = lambda x : int(x.split('\\file_')[-1]))

In [7]:
imgs = []

for i in test_img_folders:
    test_imgs = sorted(glob(i+"/*.jpg"), key = lambda x : int(x.split('.jpg')[0].split("\\")[-1]))
    for j in test_imgs[30:45]:
        imgs.append(j)

In [11]:
model = EfficientNet.from_pretrained(MODELS, num_classes=class_num, advprop=True)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

Loaded pretrained weights for efficientnet-b0


In [42]:
imgs

array(['data/new_images/test\\file_142\\30.jpg',
       'data/new_images/test\\file_142\\31.jpg',
       'data/new_images/test\\file_142\\32.jpg',
       'data/new_images/test\\file_142\\33.jpg',
       'data/new_images/test\\file_142\\34.jpg',
       'data/new_images/test\\file_142\\35.jpg',
       'data/new_images/test\\file_142\\36.jpg',
       'data/new_images/test\\file_142\\37.jpg',
       'data/new_images/test\\file_142\\38.jpg',
       'data/new_images/test\\file_142\\39.jpg',
       'data/new_images/test\\file_142\\40.jpg',
       'data/new_images/test\\file_142\\41.jpg',
       'data/new_images/test\\file_142\\42.jpg',
       'data/new_images/test\\file_142\\43.jpg',
       'data/new_images/test\\file_142\\44.jpg',
       'data/new_images/test\\file_143\\30.jpg',
       'data/new_images/test\\file_143\\31.jpg',
       'data/new_images/test\\file_143\\32.jpg',
       'data/new_images/test\\file_143\\33.jpg',
       'data/new_images/test\\file_143\\34.jpg',
       'data/new_ima

In [12]:
def predict(dataset):
    model.eval()
    result = []
    for batch_item in dataset:
        img = batch_item['img'].to(device)
        with torch.no_grad():
            output = model(img)
        output = output.cpu().numpy()
        result.extend(output)
        
    return result

In [13]:
def softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

In [14]:
answers = []
for test_folder in test_folders :
    json_path = glob(test_folder + '/*.json')[0]
    js = json.load(open(json_path))
    answers.append([test_folder.replace(data_path,'')])

answers = pd.DataFrame(answers, columns = ['test_path'])

In [33]:
save_path = 'models/Final'

In [34]:
submission = pd.read_csv(data_path + '/sample_submission.csv')
sub_ALL = []
sub_ALL_list = []

for i in tqdm(range(FOLDS)):

    model = torch.load(save_path + f"_{EPOCHS}_1.pt", map_location=device)

    test_dataset = CustomDataset(imgs=imgs, labels=None, mode='test')
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

    pred = predict(test_dataloader)

    a = []
    for i in range(len(pred)):
        a.append(softmax(pred[i]))
    a = np.array(a)
    
    sub_ALL = []
    
    for i in range(answers.shape[0]):
        sub = 0
        num_sub = 0
        for j in range(len(imgs)):
            if answers["test_path"][i] in imgs[j]:
                num_sub = num_sub + 1
                sub = sub + a[j]
        sub = sub / num_sub
        sub_ALL.append(sub)
        
    sub_ALL_list.append(sub_ALL)
    
sub_ALL_list = np.array(sub_ALL_list)
sub_ALL_list = np.mean(sub_ALL_list, axis=0)
submission.iloc[:,1:] = sub_ALL_list

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.84s/it]


In [131]:
SAVE_CSV_NAME = f'submit/Final.csv'

submission.to_csv(SAVE_CSV_NAME, index=False)